In [2]:
import pandas as pd
from transformers import pipeline
import numpy as np
from sklearn.decomposition import IncrementalPCA
import torch

2024-07-26 17:18:21.638814: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
torch.cuda.is_available()

False

In [4]:
torch.cuda.device_count()

0

In [ ]:
# Check if GPU is available
device = 0 if torch.cuda.is_available() else -1
print(device)

checkpoint = "joelniklaus/legal-xlm-longformer-base"
feature_extractor = pipeline("feature-extraction", model=checkpoint, framework="pt", device=device)  # Use GPU if available

n_components = 50  # no. comp to reduce to
batch_size = 50 
ipca = IncrementalPCA(n_components=n_components)

def extract_features(text, max_length=4096):
    if not isinstance(text, str):
        text = str(text)
    if len(text) > max_length:
        text = text[:max_length]
    features = feature_extractor(text, return_tensors="pt")
    return features[0].numpy().flatten()

def pad_or_truncate(features, target_size):
    if len(features) > target_size:
        return features[:target_size]
    elif len(features) < target_size:
        return np.pad(features, (0, target_size - len(features)), mode='constant')
    return features

def features_to_string(features):
    # Convert the numpy array to a list of strings
    return ','.join(map(str, features))

def process_batch(texts, target_size):
    batch_embeddings = [pad_or_truncate(extract_features(text), target_size) for text in texts]
    batch_embeddings = np.array(batch_embeddings)
    ipca.partial_fit(batch_embeddings)
    reduced_batch_embeddings = ipca.transform(batch_embeddings)
    
    return reduced_batch_embeddings

chunk_size = 50  # Adjust based on memory constraints
chunk_number = 0
sample_text = "This is a sample text to determine the feature size."
sample_embedding = extract_features(sample_text)
target_size = len(sample_embedding)

for chunk in pd.read_csv('dataset/dataset.csv', chunksize=chunk_size, on_bad_lines='skip'):
    reduced_features_list = []
    text_list = chunk['text'].tolist()  # Save text to ensure alignment
        for i in range(0, len(chunk), batch_size):
        batch_texts = text_list[i:i + batch_size]
        reduced_batch_features = process_batch(batch_texts, target_size)
        reduced_features_str = [features_to_string(features) for features in reduced_batch_features]
        reduced_features_list.extend(reduced_features_str)
            
    df_reduced = pd.DataFrame({
        'text': text_list[:len(reduced_features_list)],  # Ensure this matches the size, order of the reduced features
        'reduced_features': reduced_features_list
    })
    
    df_reduced.to_csv(f'dataset_with_reduced_features_chunk_{chunk_number}.csv', index=False)
    
    chunk_number += 1

print(f"Processed {chunk_number} chunks.")

-1


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of LongformerModel were not initialized from the model checkpoint at joelniklaus/legal-xlm-longformer-base and are newly initialized: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Input ids are automatically padded from 14 to 128 to be a multiple of `config.attention_window`: 128
Input ids are automatically padded from 940 to 1024 to be a multiple of `config.attention_window`: 128
Input ids are automatically padded from 911 to 1024 to be a multiple of `config.attention_window`: 128
Input ids are automatically padded from 903 to 1024 to be a multiple of `config.atte